In [1]:
import pandas as pd
import numpy as np

from underthesea import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

# from keras.layers import merge
from keras.layers import Concatenate
from keras.layers.core import Dense, Reshape
from keras.layers import Embedding
from keras.models import Sequential
from scipy import spatial

# a)	Sử dụng độ đo Bleu score

In [2]:
import os
dataPath = os.path.dirname(os.getcwd())
print(dataPath)
filename1 = os.path.join(dataPath, 'data', 'chatbot-data.csv')
filename2 = os.path.join(dataPath, 'data', 'chatbot-data-2.csv')
filename1

e:\Workspace\NodeJS\chatbot-machine-learning-nodejs


'e:\\Workspace\\NodeJS\\chatbot-machine-learning-nodejs\\data\\chatbot-data.csv'

In [3]:
# filename1 = './data/chatbot data.csv'
# filename2 = './data/chatbot data 2.csv'
combined_csv = pd.concat([pd.read_csv(f) for f in [filename1, filename2]])
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8')
df = pd.read_csv("combined_csv.csv")
print(df.head(5))
print(df.shape)
# (7380, 3)

   Unnamed: 0                                             user_a  \
0           0                                        bạn tên gì?   
1           1                   Tui tên Nam <eos> Bạn học ở đâu?   
2           2         tui học ở TDTU luôn <eos> Bạn học ngành gì   
3           3                          Chuyên ngành gì vậy bạn ?   
4           4  mình học Quản Trị Kinh Doanh <eos> Bạn là ngườ...   

                                              user_b  
0             Mình tên Trường <eos> còn bạn tên gì?   
1  tui học ở TDTU nè <eos> còn bạn học ở trường n...  
2                       mình học Công Nghệ Thông Tin  
3  mình học bên Kỹ Thuật Phần Mềm á <eos> còn bạn...  
4  Mình là người Đồng tháp <eos> bạn biết Đồng Th...  
(7380, 3)


In [4]:
# Punctuation Removal
import string

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
df['clean_user_a']= df['user_a'].apply(lambda x:remove_punctuation(x))
df['user_a_lower']= df['clean_user_a'].apply(lambda x: x.lower())
df[['user_a','clean_user_a','user_a_lower']]

,user_a,clean_user_a,user_a_lower
0,bạn tên gì?,bạn tên gì,bạn tên gì
1,Tui tên Nam <eos> Bạn học ở đâu?,Tui tên Nam eos Bạn học ở đâu,tui tên nam eos bạn học ở đâu
2,tui học ở TDTU luôn <eos> Bạn học ngành gì,tui học ở TDTU luôn eos Bạn học ngành gì,tui học ở tdtu luôn eos bạn học ngành gì
3,Chuyên ngành gì vậy bạn ?,Chuyên ngành gì vậy bạn,chuyên ngành gì vậy bạn
4,mình học Quản Trị Kinh Doanh <eos> Bạn là ngườ...,mình học Quản Trị Kinh Doanh eos Bạn là người ...,mình học quản trị kinh doanh eos bạn là người ...
...,...,...,...
7375,Nhà mày bao nhiêu người?,Nhà mày bao nhiêu người,nhà mày bao nhiêu người
7376,Có bao nhiêu anh chị em?,Có bao nhiêu anh chị em,có bao nhiêu anh chị em
7377,Mày là em út hay anh cả?,Mày là em út hay anh cả,mày là em út hay anh cả
7378,Bố mày làm nghề gì?,Bố mày làm nghề gì,bố mày làm nghề gì


In [5]:
sentences = [word_tokenize(line) for line in df['user_a_lower']]
sentences[:10]

[['bạn', 'tên', 'gì'],
 ['tui', 'tên', 'nam', 'eos', 'bạn', 'học', 'ở', 'đâu'],
 ['tui', 'học', 'ở', 'tdtu', 'luôn', 'eos', 'bạn', 'học', 'ngành', 'gì'],
 ['chuyên', 'ngành', 'gì', 'vậy', 'bạn'],
 ['mình',
  'học',
  'quản trị',
  'kinh doanh',
  'eos',
  'bạn',
  'là',
  'người',
  'ở',
  'đâu',
  'vậy'],
 ['mình', 'biết', 'đồng', 'tháp', 'dưới', 'miền', 'tây', 'đúng', 'không'],
 ['còn',
  'mình',
  'ở',
  'tphcm',
  'luôn',
  'nè',
  'mình',
  'ở',
  'quận',
  '4',
  'eos',
  'cuối',
  'tuần',
  'này',
  'bạn',
  'có',
  'rãnh',
  'không'],
 ['mình', 'tính', 'cuối', 'tuần', 'này', 'rủ', 'bạn', 'đi', 'coffee', 'nè'],
 ['đi', 'phúc', 'long', 'đối diện', 'trường', 'mình', 'nè'],
 ['được',
  'luôn',
  'bạn',
  'sáng',
  'mình',
  'cũng',
  'rãnh',
  'eos',
  'bạn',
  'thích',
  'ngôn ngữ',
  'lập trình',
  'nào',
  'nhất']]

In [6]:
import warnings
warnings.filterwarnings('ignore')

def blue(candidate):
  max_bleu = 0
  max_id = 0
  for line in range(0,len(sentences)):
    t = sentence_bleu([sentences[line]], word_tokenize(candidate))
    if t > max_bleu:
      max_bleu = t
      max_id = line
  
  return max_id

candidate = 'Cuộc sống bạn ổn không'
max_id = blue(candidate)
print(candidate)
print(df['user_b'][max_id])

Cuộc sống bạn ổn không
Cũng ổn


In [ ]:
print("**************************")
print("*    start chatting      *")
print("*    input q to exit     *")
print("**************************")
while True:
  newMessage  = input("you : ")
  if newMessage == 'q':
    print("chatbot :","tạm biệt")
    break
  max_id = blue(newMessage)
  print("chatbot :",df['user_b'][max_id])

**************************
*    start chatting      *
*    input q to exit     *
**************************
you : đang làm gì đó?
chatbot : Bấm điện thoại
you : ăn trưa không
chatbot : ok luôn
you : ăn gì
chatbot : gì cũng được
you : ăn hủ tiếu không
chatbot : không
you : ăn phở nha
chatbot : Đi anh ơi
you : ăn ở đâu
chatbot : KFC chứ đâu
you : 15 phút nữa mình sang
chatbot : Ok
you : q
chatbot : tạm biệt


In [ ]:
# **************************
# *    start chatting      *
# *    input q to exit     *
# **************************
# you : đang làm gì đó?
# chatbot : Bấm điện thoại
# you : ăn trưa không
# chatbot : ok luôn
# you : ăn gì
# chatbot : gì cũng được
# you : ăn hủ tiếu không
# chatbot : không
# you : ăn phở nha
# chatbot : Đi anh ơi
# you : ăn ở đâu
# chatbot : KFC chứ đâu
# you : 15 phút nữa mình sang
# chatbot : Ok
# you : q
# chatbot : tạm biệt

# b)	Sử dụng Word2Vec và độ đo Euclid 

In [ ]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(sentences)
word2id = t.word_index
vocab_size = len(t.word_index) + 1
print(vocab_size)
print(t.index_word)

In [28]:
wids = [[word2id[w] for w in doc] for doc in sentences]
wids[:10]

[[3, 28, 4],
 [79, 28, 139, 18, 3, 16, 7, 8],
 [79, 16, 7, 531, 136, 18, 3, 16, 80, 4],
 [912, 80, 4, 19, 3],
 [15, 16, 1162, 913, 18, 3, 13, 38, 7, 8, 19],
 [15, 61, 532, 1163, 481, 264, 672, 174, 1],
 [56, 15, 7, 914, 136, 116, 15, 7, 180, 243, 18, 119, 117, 27, 3, 2, 181, 1],
 [15, 144, 119, 117, 27, 168, 3, 5, 1591, 116],
 [5, 915, 392, 772, 32, 15, 116],
 [25, 136, 3, 265, 15, 52, 181, 18, 3, 11, 533, 916, 9, 57]]

In [29]:
from keras.preprocessing.sequence import skipgrams

# generate skip-grams
skip_grams = [skipgrams(wid, vocabulary_size=vocab_size, window_size=2) for wid in wids]
skip_grams[0]

([[4, 1544],
  [28, 3],
  [3, 4],
  [3, 28],
  [3, 2248],
  [4, 2202],
  [28, 4],
  [3, 2734],
  [28, 1088],
  [4, 3],
  [4, 28],
  [28, 1513]],
 [0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0])

In [ ]:
pairs, labels = skip_grams[0][0], skip_grams[0][1]
print(sentences[0])
for i in range(15):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          t.index_word[pairs[i][0]], pairs[i][0], 
          t.index_word[pairs[i][1]], pairs[i][1], 
          labels[i]))

In [31]:
from keras.layers import Input
from keras.models import Model
# from keras.layers.embeddings import Embedding
from keras.layers import Embedding
from keras.layers.core import Dense, Reshape
from keras.layers import dot

embed_size = 400
input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, embed_size, input_length=1, name='embedding')

word_embedding = embedding(input_target)
word_embedding = Reshape((embed_size, 1))(word_embedding)
context_embedding = embedding(input_context)
context_embedding = Reshape((embed_size, 1))(context_embedding)
 
dot_product = dot([word_embedding, context_embedding], axes=1)
dot_product = Reshape((1,))(dot_product)

output = Dense(1, activation='sigmoid')(dot_product)

model = Model(inputs=[input_target, input_context], outputs=output)
model.compile(loss='mean_squared_error', optimizer='rmsprop')

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 400)       1199200     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 400, 1)       0           ['embedding[0][0]']          

In [32]:
for epoch in range(0, 10):
    loss = 0
    for i, elem in enumerate(skip_grams):
        # print('i = ',i,'elem = ',elem)
        if elem[0] == []:
          continue
        pair_first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        pair_second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        labels = np.array(elem[1], dtype='int32')
        X = [pair_first_elem, pair_second_elem]
        Y = labels

        loss += model.train_on_batch(X,Y)  

    print('Epoch:', epoch, 'Loss:', loss)

Epoch: 0 Loss: 1264.8110978108598
Epoch: 1 Loss: 1149.3945701805205
Epoch: 2 Loss: 1116.1033267735038
Epoch: 3 Loss: 1104.1304842089394
Epoch: 4 Loss: 1099.0400937302402
Epoch: 5 Loss: 1096.675175725044
Epoch: 6 Loss: 1095.736146253812
Epoch: 7 Loss: 1095.5241110866045
Epoch: 8 Loss: 1095.7201200372501
Epoch: 9 Loss: 1096.2134937361075


In [34]:
T = model.layers[2]
weights = T.get_weights()[0][1:]

print(weights.shape)
df_embedding = pd.DataFrame(weights, index=t.index_word.values())
df_embedding

(2997, 400)


,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
không,-0.043438,2.460308,0.160131,0.324053,0.150417,-0.721172,-2.297697,-0.016855,-1.020152,4.272995,...,0.166825,-4.889871,4.205751,-1.160404,-0.752154,-0.097502,0.776252,1.146243,0.155308,-3.910745
có,-1.176667,1.980563,0.070527,1.323157,0.598634,0.177911,-0.783718,-0.362770,-0.465439,1.046377,...,1.506529,0.853487,2.407836,0.023161,-0.466386,0.456006,0.619772,-0.786494,3.646927,0.562335
bạn,0.332893,0.869873,-0.136999,-1.186898,0.503483,-0.629861,-2.116546,-0.054564,-1.601541,-1.852975,...,2.270709,1.834157,-0.361105,-1.759112,0.321802,1.027301,-0.985262,1.170206,-0.517773,2.969092
gì,-1.183464,2.265480,-0.532594,-1.611488,-0.571018,-0.458084,-2.499362,-1.338778,-2.193396,2.932612,...,2.688513,3.148631,3.191689,-1.070627,-1.553925,1.651627,-0.290350,1.443407,-0.577872,-2.257808
đi,-1.198121,0.458358,-1.016319,-1.509644,1.309836,2.057400,-2.398912,-0.183441,-0.575773,-0.926128,...,-1.369696,-1.424966,3.683705,-1.135950,-0.528981,0.476215,-0.863703,0.673559,-1.120610,3.915354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ruột,0.765082,-0.577671,0.641848,0.701105,-0.610467,-0.571837,0.672568,0.652741,0.605512,0.151478,...,-0.499545,-0.320690,-0.565621,0.532408,0.615499,-0.751732,0.618714,-0.608113,0.548060,-0.119786
nội ngoại,0.622449,-0.580397,0.558366,0.389840,-0.548025,-0.515072,0.488760,0.560669,0.407168,-0.193952,...,-0.308372,0.108329,-0.584959,0.404432,0.524113,-0.421198,0.503982,-0.577119,0.367650,0.159169
bme,0.579892,-0.348380,0.538794,0.457278,-0.516321,-0.646065,0.262232,0.486577,0.204033,0.085776,...,-0.153335,-0.332847,-0.216834,0.409946,0.514130,-0.338769,0.643268,-0.536628,0.697858,0.251932
vô duyên,0.602243,-0.717017,0.582823,0.657418,-0.680196,-0.437127,0.671643,0.447871,0.701020,0.256335,...,-0.614583,-0.301262,-0.412846,0.651722,0.501917,-0.564826,0.765456,-0.619066,0.542056,-0.130712


In [ ]:
def compare(query):
  max_cs = 0
  max_id = 0
  query = remove_punctuation(query)
  query = query.lower()
  query = word_tokenize(query)

  query_arr = []
  for token in query:
    try:
      query_arr.append(list(df_embedding.loc[token]))
    except BaseException:
      pass 
  query_arr = np.array(query_arr)
  sum_query_arr = np.sum(query_arr,0)


  for i in range(0,len(sentences)):
    M = np.array([list(df_embedding.loc[token]) for token in sentences[i]])
    sum_arr = np.sum(M,0)

    #compare sentence(M) and query
    cs = 1 - spatial.distance.cosine(sum_query_arr, sum_arr)
    if cs > max_cs:
      max_cs = cs
      max_id = i
  
  return max_id

max_id = compare('Bạn học trường nào?')
df['user_b'][max_id]

In [ ]:
print("**************************")
print("*    start chatting      *")
print("*    input q to exit     *")
print("**************************")
while True:
  newMessage  = input("you : ")
  if newMessage == 'q':
    print("chatbot :","tạm biệt")
    break
  max_id = compare(newMessage)
  print("chatbot :",df['user_b'][max_id])

**************************
*    start chatting      *
*    input q to exit     *
**************************
you : Bạn học trường nào
chatbot : Tớ học đại học Tôn Đức Thắng
you : Bạn bao nhiêu tuổi?
chatbot : mình 21 tuổi
you : Bạn bao nhiêu tuổi?
chatbot : mình 21 tuổi
you : Bạn có sở thích gì?
chatbot : Núi và sông.
you : có chơi thể thao không
chatbot : Có.
you : chiều đi đá banh không
chatbot : đi
you : ok
chatbot : chúc ngày mới vui vể
you : q
chatbot : tạm biệt


In [ ]:
# **************************
# *    start chatting      *
# *    input q to exit     *
# **************************
# you : Bạn học trường nào
# chatbot : Tớ học đại học Tôn Đức Thắng
# you : Bạn bao nhiêu tuổi?
# chatbot : mình 21 tuổi
# you : Bạn bao nhiêu tuổi?
# chatbot : mình 21 tuổi
# you : Bạn có sở thích gì?
# chatbot : Núi và sông.
# you : có chơi thể thao không
# chatbot : Có.
# you : chiều đi đá banh không
# chatbot : đi
# you : ok
# chatbot : chúc ngày mới vui vể
# you : q
# chatbot : tạm biệt

# Server

In [7]:
from flask import Flask, request
import json

# Setup flask server
app = Flask(__name__)

# Setup url route which will calculate
@app.route('/chatbot', methods = ['POST'])
def chat():
	data = request.get_json()
	print(data)

	max_id = blue(data)
	result = df['user_b'][max_id]
	print(result)

	# Return data in json format
	return json.dumps({"result":result})

if __name__ == "__main__":
	app.run(port=5000)

# npx kill-port 5000

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


chào bạn


127.0.0.1 - - [27/Feb/2023 08:02:26] "POST /chatbot HTTP/1.1" 200 -


Xin chào
chào bạn


...

Xin chào
đi chơi không


127.0.0.1 - - [27/Feb/2023 16:06:00] "POST /chatbot HTTP/1.1" 200 -


đi
chào bạn


127.0.0.1 - - [27/Feb/2023 16:07:29] "POST /chatbot HTTP/1.1" 200 -


Xin chào
